In [3]:
%matplotlib inline
from bs4 import BeautifulSoup
import urllib2
import urllib
import json
import csv
import time
import pickle
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
import sys
import os
import time
sns.set_style("whitegrid")
sns.set_context("poster")

In [2]:
def load_obj(name):
    with open(name + '.pkl', 'rb') as f:
        return pickle.load(f)
    
def merge_d(x, y):
    z = x.copy()
    z.update(y)
    return z

def flatten_data(t_data):
    t_data_f= {}
    for key in t_data.keys():
        for key2 in t_data[key].keys():
            t_data_f[key + '_' + key2] = t_data[key][key2]
    return t_data_f

twit_data = flatten_data(load_obj('twit_data1'))
t_data_num = 1

def backup(obj, t_data_num = t_data_num):
    name = "t_df" + str(t_data_num)
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

We now want to do some data wrangling to aggregate all the data from the Tweets. We want to get information about the tweet text, the number of likes, the number of retweets, names, and href. Later on, we can extract more analytics about these types of interactions.

In [12]:
company_l = []
series_l = []
text_l = [] 
time_l = []
retweets_l = []
likes_l = []
name_l = []
href_l = []
errors = []
max_ind = len(twit_data.keys())
start = time.time()

for i in range(max_ind):    
    
    # Timing code and backup code
    if i % 50 == 0:
        mins = (time.time() - start) / 60
        print 'Percent progress ' + str(100.0 * i / max_ind) + ' running for ' + str(mins) + ' mins'          
        data_backup = {'Company': company_l, 'Series': series_l, 'Text': text_l, 'Time':time_l, 'Retweets':retweets_l, 'Likes':likes_l, 'Name':name_l, 'Href':href_l}
        backup(data_backup)
    
    # Finding the series of the data
    company_series = twit_data.keys()[i]
    ind = company_series[:company_series.rfind("_")].rfind("_")
    company = company_series[:ind]
    series = company_series[ind:]
    backup({'Company': company_l, 'Series': series_l, 'Text': text_l, 'Time':time_l, 'Retweets':retweets_l, 'Likes':likes_l, 'Name':name_l, 'Href':href_l})
    
    
    # Loop through each copmany and series pair and extract relevant information
    for tweet in twit_data[company_series]:
        try:
            # Searching
            soup = BeautifulSoup(tweet, 'lxml')
            actions = soup.find_all('div', class_='ProfileTweet-actionCountList')[0]
            text = soup.find_all('p', class_ = 'TweetTextSize')[0].text
            times = soup.find_all('a', class_ = 'tweet-timestamp')[0].text
            retweets = actions.find_all('span', class_='ProfileTweet-actionCount')[0].text.strip()
            likes = actions.find_all('span', class_='ProfileTweet-actionCount')[1].text.strip()
            name = soup.find_all('strong', class_='fullname')[0].text
            href = soup.find_all('a', class_='account-group')[0]['href']
            
            # Storing
            company_l.append(company)
            series_l.append(series)
            text_l.append(text)
            time_l.append(times)
            retweets_l.append(retweets)
            likes_l.append(likes)
            name_l.append(name)
            href_l.append(href)
            
        # Only listen for keyboard exceptions
        except KeyboardInterrupt:
            try:
                sys.exit(0)
            except SystemExit:
                os._exit(0) 
        except:
            errors.append(tweet)

Percent progress 0.0 running for 5.08666038513e-05 mins
Percent progress 5.0 running for 0.929983933767 mins
Percent progress 10.0 running for 2.19964901606 mins
Percent progress 15.0 running for 3.53876201709 mins
Percent progress 20.0 running for 4.88092761834 mins
Percent progress 25.0 running for 6.11316250165 mins
Percent progress 30.0 running for 7.68281498353 mins
Percent progress 35.0 running for 9.68449503183 mins
Percent progress 40.0 running for 11.6382258654 mins
Percent progress 45.0 running for 13.2810905496 mins
Percent progress 50.0 running for 15.5552463651 mins
Percent progress 55.0 running for 17.8359643817 mins
Percent progress 60.0 running for 20.021214668 mins
Percent progress 65.0 running for 22.598205018 mins
Percent progress 70.0 running for 25.0689711491 mins
Percent progress 75.0 running for 28.2786712845 mins
Percent progress 80.0 running for 31.1675331672 mins
Percent progress 85.0 running for 34.0141220649 mins
Percent progress 90.0 running for 37.26210173

After extracting all tweets from the HTML page, we now want to try to generate features that we can regress on. We save the tweets to a file, and then extract features from all of these tweets.

In [5]:
tweets = pd.read_csv(open("../tweets/tweets.csv", 'rU'), encoding='utf-8', engine='c')
tweets_translated = pd.read_csv(open("../startups/translatedTweets.csv", 'rU'), encoding='utf-8', engine='c')
tweets_translated.head()

/Users/candokevin/anaconda/lib/python2.7/site-packages/pandas/io/parsers.py:1170: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


,Unnamed: 0,Unnamed: 0.1,Company,Href,Likes,Name,Retweets,Series,Text,Time
0,0,0,Occipital,/yehyehgluglu,0 likes,Maria Melo,0 retweets,_Series_A,@rubaums2 mira in the occipital and frontal to...,14 Jul 2011
1,1,2,Occipital,/ArturoMisifu,0 likes,Arturo Quijano,0 retweets,_Series_A,at the height of the occipital head - hurts if...,14 Jul 2011
2,2,5,Occipital,/victorcab,0 likes,Victor Caballero,0 retweets,_Series_A,Tripod – Timer for 360 Views – iPhone? : http:...,14 Jul 2011
3,3,7,Occipital,/electroniko,0 likes,MIGUEL NEGRETE,0 retweets,_Series_A,"Neurona, glia, axon myelin, parietal, temporal...",14 Jul 2011
4,4,8,Occipital,/nomina_bot,0 likes,体野ミナ,0 retweets,_Series_A,Greater occipital nerve:nervus occipitalis maj...,13 Jul 2011


We only take tweets that relate to companies that have 150 or more tweets. We also want to calculate the average number of days it took to reach 200 tweets. 

In [ ]:
# Calculating relative dates based off of dates like Apr, 5 2015. We can use this later
# for calculating average number of tweets per day.
months = {'Jan':1, 'Feb':2, 'Mar':3, 'Apr':4, 'May':5, 'Jun':6, 'Jul':7, 'Aug':8, 'Sep':9, 'Oct':10, 'Nov':11, 'Dec':12}
time_sub = []
for row in tweets.iterrows():
    if type(row[1]['Time']) != unicode:
        time_sub.append(0)
    elif len(row[1]['Time'].split(' ')) == 3:
        if row[1]['Time'].split(' ')[1] in months.keys():        
            time_sub.append(int(row[1]['Time'].split(' ')[2]) * 365 + months[row[1]['Time'].split(' ')[1]] * 30 + int(row[1]['Time'].split(' ')[0]))
        else:
            time_sub.append(0)
    else:
        time_sub.append(0)
        
tweets['Date_Num'] = time_sub
tweets = tweets[tweets['Date_Num'] != 0]

valid_data = []
for val in zip(list((tweets.groupby(['Company', 'Series']).size() > 150).index), list((tweets.groupby(['Company', 'Series']).size() > 150).values)):
    if val[1]:
        valid_data.append(val[0])
        
# We also only filter the tweets based off of whether or not there was 150 tweets in the past
valid_tweets = []
for valid_pair in valid_data:
    company = valid_pair[0]
    series = valid_pair[1]
    match_company = tweets[tweets['Company'] == company]
    valid_tweets.append(match_company[match_company['Series'] == series])


In [69]:
# We are now incorporate the translated tweets that Roger generated
valid_tweets_pd = pd.concat(valid_tweets)
for row in tweets_translated.iterrows():
    company = row[1]['Company']
    href = row[1]['Href']
    time = row[1]['Time']

    match_comp = valid_tweets_pd[valid_tweets_pd['Company'] == company]
    match_href = match_comp[match_comp['Href'] == href]
    match_time = match_href[match_href['Time'] == time]
    
    res = list(match_time.index)
    if len(res) == 1:
        index = res[0]
        valid_tweets_pd.set_value(index, 'Text', row[1].Text)

In [71]:
valid_tweets_pd.to_csv('valid_tweets.csv', encoding='utf-8')

In [68]:
len(tweets_translated)

198201

It seems that the total number of tweets is now 2/3 of what it used to be. It seems reasonable to drop this many tweets.

In [ ]:
valid_tweets_pd = pd.concat(valid_tweets)
valid_tweets_pd['Likes'] = valid_tweets_pd.apply(lambda row: int(row['Likes'][0]), axis=1)
valid_tweets_pd['Retweets'] = valid_tweets_pd.apply(lambda row: int(row['Retweets'][0]), axis=1)

Now we want to extract relevant features like the number of likes, retweets, and other relevant features for each company funding round pairing. 

In [224]:
# Calculate average tweets
days = valid_tweets_pd.groupby(['Company', 'Series'])['Date_Num'].max() - valid_tweets_pd.groupby(['Company', 'Series'])['Date_Num'].min()
tweets = valid_tweets_pd.groupby(['Company', 'Series']).size()
avg_tweets = tweets / days

# Calculate total likes
total_likes= valid_tweets_pd.groupby(['Company', 'Series'])['Likes'].sum()

# Calculate total retweets
total_retweets = valid_tweets_pd.groupby(['Company', 'Series'])['Retweets'].sum()
companies = [val[0] for val in list(tweets.index)]
series = [val[1] for val in list(tweets.index)]

# Calculate all features
tweet_features = pd.DataFrame({'Index': range(1, 1+len(days)), 'Company':companies, 'Series':series, 'Total_Likes': total_likes, 'Total_Retweets': total_retweets, 'Avg_Tweets': avg_tweets})
tweet_features.index = range(1, 1+len(days))

# Open old funding data
funding_df=pd.read_csv("funding_collapsed.csv")
funding_df = funding_df.drop(['Unnamed: 0'], 1)
funding_df.rename(columns={'Names':'Company', 'Series_Type':'Series'}, inplace=True)

# Merge the tweet features with the funding data
regress_features = pd.merge(tweet_features, funding_df, how='left', on=['Company', 'Series'])
regress_features = regress_features.drop('Index', 1)
regress_features = regress_features[([type(amount) == str for amount in list(regress_features.Series_Amount)])]
regress_features['Series_Amount'] = regress_features.apply(lambda row: int(row['Series_Amount'][1:].replace(',','')), axis=1)
regress_features.to_csv('regress_features.csv')